In [ ]:
import cv2
import numpy as np

In [ ]:
def save_initial():
    '''Used to save initial picture of the speedometer to be later used for different computations. 
    Saves a "reference.jpg" file in the current working directory'''
    try:
        cap = cv2.VideoCapture(0)
        while True:
            _,frame = cap.read()
            frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            frame_gray_copy = np.copy(frame_gray)
            cv2.putText(frame_gray_copy, "Press 's' to save image", (0,25), 2, 0.9, (255,255,255), 1, cv2.LINE_AA, False)
            cv2.line(frame_gray_copy, (0,30), (frame_gray_copy.shape[1], 30), (255,255,255))
            cv2.imshow('frame', frame_gray_copy)
            k = cv2.waitKey(1) & 0xFF
            if k in [115, 83]:
                cv2.imwrite('reference.jpg', frame_gray)
                print('Image Saved')
                break
            elif k == 27:
                break
    except:
        print('An error has occurred.')
    cap.release()
    cv2.destroyAllWindows()

In [ ]:
def compute_sum_thresh(frame, tolerance_percent=5):
    return_value = {'pixel_sum':-1, 'lower_limit':-1, 'upper_limit':-1}
    return_value['pixel_sum'] = np.sum(frame/ frame.size)
    return_value['lower_limit'] = return_value['pixel_sum']*(1-tolerance_percent/100)
    return_value['upper_limit'] = return_value['pixel_sum']*(1+tolerance_percent/100)
    return return_value

In [ ]:
def compare(filename='reference.jpg'):
    tolerance_percent = 5
    # Read the image
    reference_img = cv2.imread(filename, 0)
    
    ref_img_values = compute_sum_thresh(reference_img, tolerance_percent)
    cv2.imshow('Reference Image', reference_img)
    
    print("\nDisplaying reference image.\nPress 'Esc' to exit, any other key to continue.")
    k = cv2.waitKey(0)
    if k == 27:
        cv2.destroyAllWindows()
        return
    
    cap = cv2.VideoCapture(0)
    while True:
        _, frame = cap.read()
        frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        frame_values = compute_sum_thresh(frame_gray, tolerance_percent)
        
        okng_text = ""
        okng_color = (0,0,0)
        if frame_values['pixel_sum'] > ref_img_values['upper_limit'] or frame_values['pixel_sum'] < ref_img_values['lower_limit']:
            okng_text = 'NG'
            okng_color = (0,0,255)
        else:
            okng_text = 'OK'
            okng_color = (0,255,0)
        
        blackpart = np.zeros((30, frame_gray.shape[1]), dtype='uint8')
        text_blackpart1 = "Desired: [" + str(ref_img_values['lower_limit']) + ', '+ str(ref_img_values['upper_limit']) +']'
        text_blackpart2 = "Current: " + str(frame_values['pixel_sum'])
        
        frame_gray = np.vstack((frame_gray, blackpart))
        
        cv2.putText(frame_gray,text_blackpart1+' '*10+text_blackpart2,(10,frame_gray.shape[0]-10),2,0.4,(255,255,255),1, cv2.LINE_AA)
        
        frame_output = cv2.cvtColor(frame_gray, cv2.COLOR_GRAY2BGR)
        cv2.putText(frame_output, okng_text, (10,25), 2, 1.2, okng_color, 1, cv2.LINE_AA)
        
        print(text_blackpart1, '\t', text_blackpart2)
        cv2.imshow('Output', frame_output)
        
        k = cv2.waitKey(1) & 0xFF
        if k == 27:
            cap.release()
            break
    cv2.destroyAllWindows()

In [ ]:
save_initial()

In [ ]:
compare()